READ
DOI: https://archive.ics.uci.edu/dataset/327/phishing+websites



In [3]:
import arff
import pandas as pd
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, mutual_info_classif

# ========== Đường dẫn ==========
BASE_DIR = os.getcwd()
DATA_PATH = os.path.join(BASE_DIR, "phishing.arff")

# ========== Đọc dữ liệu từ file .arff ==========
with open(DATA_PATH, "r") as f:
    data = arff.load(f)

columns = [col[0] for col in data["attributes"]]
df = pd.DataFrame(data["data"], columns=columns).astype(int)

# ========== Tổng quan ==========
print("🧾 Tổng quan về Dataset:")
print(f"- Số dòng (samples): {df.shape[0]}")
print(f"- Số thuộc tính (features): {df.shape[1] - 1}")
print("\n📌 Danh sách các thuộc tính:")
for i, col in enumerate(df.columns[:-1]):
    print(f"{i+1:2d}. {col}")

# ========== Phân tích đặc trưng ==========
X = df.drop("Result", axis=1)
y = df["Result"]

# Dùng RandomForest để đánh giá mức độ quan trọng
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X, y)
importances = rf.feature_importances_

# Tạo DataFrame để hiển thị rõ hơn
importance_df = pd.DataFrame({
    "Feature": X.columns,
    "Importance": importances
}).sort_values(by="Importance", ascending=False)

print("\n🔥 10 đặc trưng quan trọng nhất (theo RandomForest):")
print(importance_df.head(10).to_string(index=False))

# ========== Top đặc trưng theo Mutual Information ==========
selector = SelectKBest(score_func=mutual_info_classif, k=10)
selector.fit(X, y)
mi_scores = pd.DataFrame({
    "Feature": X.columns,
    "MI Score": selector.scores_
}).sort_values(by="MI Score", ascending=False)

print("\n💡 10 đặc trưng quan trọng nhất (theo Mutual Information):")
print(mi_scores.head(10).to_string(index=False))


🧾 Tổng quan về Dataset:
- Số dòng (samples): 11055
- Số thuộc tính (features): 30

📌 Danh sách các thuộc tính:
 1. having_IP_Address
 2. URL_Length
 3. Shortining_Service
 4. having_At_Symbol
 5. double_slash_redirecting
 6. Prefix_Suffix
 7. having_Sub_Domain
 8. SSLfinal_State
 9. Domain_registeration_length
10. Favicon
11. port
12. HTTPS_token
13. Request_URL
14. URL_of_Anchor
15. Links_in_tags
16. SFH
17. Submitting_to_email
18. Abnormal_URL
19. Redirect
20. on_mouseover
21. RightClick
22. popUpWidnow
23. Iframe
24. age_of_domain
25. DNSRecord
26. web_traffic
27. Page_Rank
28. Google_Index
29. Links_pointing_to_page
30. Statistical_report

🔥 10 đặc trưng quan trọng nhất (theo RandomForest):
                    Feature  Importance
             SSLfinal_State    0.318529
              URL_of_Anchor    0.262463
                web_traffic    0.070082
          having_Sub_Domain    0.060848
              Links_in_tags    0.041492
              Prefix_Suffix    0.038782
                

Nhìn nhanh ta thấy đây là bài toán phân lớp dạng nhị phân. Website có phải là lừa đảo (phishing) hay không?

MÔ TẢ DỮ LIỆU:
Có 30 thuộc tính và các đặc trưng (nhãn) rời rạc:
-1: phishing Lừa đảo
1: legitimate Hợp pháp
0: suspicious Khả nghi

THƯ VIỆN SỬ DỤNG:
- Decision Tree Cây quyết định.
- Random Forest Rừng ngẫu nhiên. Có Bagging
- K-Nearest Neighbors 

1. Giới thiệu
2. Cơ sở lý thuyết Machine Learning
3. Dữ liệu và xử lý
4. Mô hình và huấn luyện
5. Đánh giá kết quả
6. Ứng dụng vào thực tế
7. Kết luận và hướng phát triển
8. Phụ lục: Code, demo

In [4]:
df.head()

,having_IP_Address,URL_Length,Shortining_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,having_Sub_Domain,SSLfinal_State,Domain_registeration_length,Favicon,...,popUpWidnow,Iframe,age_of_domain,DNSRecord,web_traffic,Page_Rank,Google_Index,Links_pointing_to_page,Statistical_report,Result
0,-1,1,1,1,-1,-1,-1,-1,-1,1,...,1,1,-1,-1,-1,-1,1,1,-1,-1
1,1,1,1,1,1,-1,0,1,-1,1,...,1,1,-1,-1,0,-1,1,1,1,-1
2,1,0,1,1,1,-1,-1,-1,-1,1,...,1,1,1,-1,1,-1,1,0,-1,-1
3,1,0,1,1,1,-1,-1,-1,1,1,...,1,1,-1,-1,1,-1,1,-1,1,-1
4,1,0,-1,1,1,-1,1,1,-1,1,...,-1,1,-1,-1,0,-1,1,1,1,1
